In [1]:
import os

In [2]:
pip install gitpython

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install langchain-community

Note: you may need to restart the kernel to use updated packages.


In [5]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [6]:
%pwd

'c:\\Users\\This PC\\source-code-detector-using-GENAI\\research'

In [7]:
!mkdir test_repo

A subdirectory or file test_repo already exists.


In [9]:
repo_path = "test_repo/"

Repo.clone_from("https://github.com/poonamDS1702/source-code-detector-using-GENAI", to_path=repo_path)

GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/poonamDS1702/source-code-detector-using-GENAI test_repo/
  stderr: 'fatal: destination path 'test_repo' already exists and is not an empty directory.
'

In [10]:
repo_path = "test_repo/"

loader = GenericLoader.from_filesystem(repo_path+'/src/mlProject',
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [11]:
documents = loader.load()

In [12]:
documents

[]

In [13]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 2000,
                                                             chunk_overlap = 200)

In [14]:
texts = documents_splitter.split_documents(documents)

In [15]:
len(texts)

0

In [16]:
pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [17]:
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()
huggingface_api_token = os.getenv("HUGGINGFACE_API_KEY")

In [18]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [19]:
from langchain.embeddings import HuggingFaceEmbeddings

# Load Hugging Face embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\This PC\AppData\Local\Temp\ipykernel_16208\1374681304.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\This PC\anaconda3\envs\llmapp\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [20]:
# vector DB

In [21]:
pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [22]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./data')
vectordb.persist()

ValueError: Expected Embedings to be non-empty list or numpy array, got [] in upsert.

In [ ]:
from langchain.chat_models import HuggingFaceHub
from langchain.memory import ConversationSummaryMemory
import os

# Set Hugging Face API Key
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_hf_api_key"

# Use a Hugging Face chat model (e.g., Mistral-7B)
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct",
    model_kwargs={"temperature": 0.7, "max_length": 512}
)

# Memory for conversation
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)


In [ ]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":3}), memory=memory)

In [ ]:
question = "what is DataIngestion class?"

In [ ]:
result = qa(question)
print(result['answer'])